# Notebook 05 — Stress Testing

## How Fragile Is Each Currency Segment Under Stress?

Apply hypothetical shocks to interest rate spreads and examine how CRI responds.

**Stress scenarios:**
- Mild (δ = 0.10): 10% increase in spreads
- Moderate (δ = 0.30): 30% increase in spreads
- Severe (δ = 0.50): 50% increase in spreads

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats as sp_stats
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.figsize': (12, 6), 'figure.dpi': 150, 'savefig.dpi': 300,
    'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12,
    'legend.fontsize': 10, 'font.family': 'serif'
})
print('Libraries loaded.')

In [ ]:
# ─── Load Data ───────────────────────────────────────────────────────────────
usd = pd.read_csv('../data/processed/spreads_usd_new_amount.csv', parse_dates=['date'], index_col='date')
khr = pd.read_csv('../data/processed/spreads_khr_new_amount.csv', parse_dates=['date'], index_col='date')
S_usd = usd['spread'].values
S_khr = khr['spread'].values
dt = 1/12

# OU estimation function
def ou_neg_log_likelihood(params, data, dt):
    kappa, theta, sigma = params
    if kappa <= 0 or sigma <= 0:
        return 1e10
    n = len(data) - 1
    exp_kdt = np.exp(-kappa * dt)
    m = theta + (data[:-1] - theta) * exp_kdt
    v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
    if v <= 0:
        return 1e10
    residuals = data[1:] - m
    ll = -0.5 * n * np.log(2 * np.pi) - 0.5 * n * np.log(v) - 0.5 * np.sum(residuals**2) / v
    return -ll

def estimate_ou(data, dt):
    best_nll = np.inf
    best_x = None
    for k0 in [0.5, 1.0, 2.0, 5.0, 10.0]:
        result = minimize(ou_neg_log_likelihood,
                         [k0, np.mean(data), np.std(np.diff(data))*np.sqrt(12)],
                         args=(data, dt), method='Nelder-Mead',
                         options={'maxiter': 50000, 'xatol': 1e-10, 'fatol': 1e-10})
        if result.fun < best_nll and result.x[0] > 0 and result.x[2] > 0:
            best_nll = result.fun
            best_x = result.x
    return {'kappa': best_x[0], 'theta': best_x[1], 'sigma': best_x[2]}

def compute_cri(data, kappa, theta, sigma, Sc, dt):
    n = len(data)
    prob = np.zeros(n)
    exp_kdt = np.exp(-kappa * dt)
    v = (sigma**2 / (2 * kappa)) * (1 - np.exp(-2 * kappa * dt))
    v_uncond = sigma**2 / (2 * kappa)
    prob[0] = 1 - sp_stats.norm.cdf(Sc, loc=theta, scale=np.sqrt(v_uncond))
    for t in range(1, n):
        m_t = theta + (data[t-1] - theta) * exp_kdt
        prob[t] = 1 - sp_stats.norm.cdf(Sc, loc=m_t, scale=np.sqrt(v))
    return prob.mean()  # average CRI probability component

print('Functions defined. Ready for stress testing.')

In [ ]:
# ─── Baseline Estimation ─────────────────────────────────────────────────────
baseline_usd = estimate_ou(S_usd, dt)
baseline_khr = estimate_ou(S_khr, dt)

Sc_usd = np.percentile(S_usd, 95)
Sc_khr = np.percentile(S_khr, 95)

sigma_max = max(baseline_usd['sigma'], baseline_khr['sigma']) * 1.5

def full_cri(data, params, Sc, dt):
    prob = compute_cri(data, params['kappa'], params['theta'], params['sigma'], Sc, dt)
    sigma_norm = params['sigma'] / sigma_max
    return 0.5 * sigma_norm + 0.5 * prob

cri_baseline_usd = full_cri(S_usd, baseline_usd, Sc_usd, dt)
cri_baseline_khr = full_cri(S_khr, baseline_khr, Sc_khr, dt)
cri_baseline_sys = 0.80 * cri_baseline_usd + 0.20 * cri_baseline_khr

print(f'Baseline CRI — USD: {cri_baseline_usd:.4f}, KHR: {cri_baseline_khr:.4f}, System: {cri_baseline_sys:.4f}')

In [ ]:
# ─── Stress Testing ──────────────────────────────────────────────────────────
deltas = [0.10, 0.30, 0.50]
delta_names = ['Mild (δ=0.10)', 'Moderate (δ=0.30)', 'Severe (δ=0.50)']

results = [{
    'Scenario': 'Baseline',
    'δ': 0.0,
    'CRI_USD': cri_baseline_usd,
    'CRI_KHR': cri_baseline_khr,
    'CRI_System': cri_baseline_sys,
    'σ_USD': baseline_usd['sigma'],
    'σ_KHR': baseline_khr['sigma'],
    'θ_USD': baseline_usd['theta'],
    'θ_KHR': baseline_khr['theta'],
}]

for delta, dname in zip(deltas, delta_names):
    print(f'\nStress scenario: {dname}')
    
    # Stressed spreads
    S_usd_stress = S_usd * (1 + delta)
    S_khr_stress = S_khr * (1 + delta)
    
    # Re-estimate parameters on stressed data
    stress_usd = estimate_ou(S_usd_stress, dt)
    stress_khr = estimate_ou(S_khr_stress, dt)
    
    # Use stressed thresholds (95th percentile of stressed data)
    Sc_usd_s = np.percentile(S_usd_stress, 95)
    Sc_khr_s = np.percentile(S_khr_stress, 95)
    
    cri_usd_s = full_cri(S_usd_stress, stress_usd, Sc_usd_s, dt)
    cri_khr_s = full_cri(S_khr_stress, stress_khr, Sc_khr_s, dt)
    cri_sys_s = 0.80 * cri_usd_s + 0.20 * cri_khr_s
    
    results.append({
        'Scenario': dname,
        'δ': delta,
        'CRI_USD': cri_usd_s,
        'CRI_KHR': cri_khr_s,
        'CRI_System': cri_sys_s,
        'σ_USD': stress_usd['sigma'],
        'σ_KHR': stress_khr['sigma'],
        'θ_USD': stress_usd['theta'],
        'θ_KHR': stress_khr['theta'],
    })
    
    print(f'  CRI — USD: {cri_usd_s:.4f} (Δ: {cri_usd_s - cri_baseline_usd:+.4f}), '
          f'KHR: {cri_khr_s:.4f} (Δ: {cri_khr_s - cri_baseline_khr:+.4f}), '
          f'System: {cri_sys_s:.4f} (Δ: {cri_sys_s - cri_baseline_sys:+.4f})')

In [ ]:
# ─── TABLE 4: Stress Test Results ────────────────────────────────────────────
stress_df = pd.DataFrame(results)

# Add delta columns
stress_df['Δ_CRI_USD'] = stress_df['CRI_USD'] - cri_baseline_usd
stress_df['Δ_CRI_KHR'] = stress_df['CRI_KHR'] - cri_baseline_khr
stress_df['Δ_CRI_System'] = stress_df['CRI_System'] - cri_baseline_sys

display_cols = ['Scenario', 'CRI_USD', 'CRI_KHR', 'CRI_System', 'Δ_CRI_USD', 'Δ_CRI_KHR', 'Δ_CRI_System']
print('\n═══════════════════════════════════════════════════════════════════════════════')
print('                    TABLE 4: Stress Test Results')
print('═══════════════════════════════════════════════════════════════════════════════')
print(stress_df[display_cols].set_index('Scenario').round(4).to_string())
print('═══════════════════════════════════════════════════════════════════════════════')

In [ ]:
# ─── FIGURE 11: Stress Test Comparison ───────────────────────────────────────
fig, ax = plt.subplots(figsize=(10, 6))

scenarios = [r['Scenario'] for r in results]
x = np.arange(len(scenarios))
width = 0.25

bars1 = ax.bar(x - width, [r['CRI_USD'] for r in results], width,
               label='CRI (USD)', color='#1565C0', alpha=0.85)
bars2 = ax.bar(x, [r['CRI_KHR'] for r in results], width,
               label='CRI (KHR)', color='#C62828', alpha=0.85)
bars3 = ax.bar(x + width, [r['CRI_System'] for r in results], width,
               label='CRI (System)', color='#4A148C', alpha=0.85)

# Value labels
for bars in [bars1, bars2, bars3]:
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords='offset points',
                    ha='center', va='bottom', fontsize=8)

ax.set_xlabel('Stress Scenario')
ax.set_ylabel('Average CRI')
ax.set_title('Figure 11: Stress Test — CRI Under Different Shock Scenarios',
             fontweight='bold', fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels(scenarios, fontsize=9)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../figures/fig11_stress_test.png', dpi=300, bbox_inches='tight')
plt.show()
print('Saved: fig11_stress_test.png')

In [ ]:
# ─── Key Finding ─────────────────────────────────────────────────────────────
usd_sensitivity = (results[-1]['CRI_USD'] - cri_baseline_usd) / cri_baseline_usd * 100
khr_sensitivity = (results[-1]['CRI_KHR'] - cri_baseline_khr) / cri_baseline_khr * 100

print(f'\n═══════════════════════════════════════════════════════')
print(f'  Which currency is more fragile under stress?')
print(f'═══════════════════════════════════════════════════════')
print(f'  Under severe stress (δ=0.50):')
print(f'  USD CRI changed by {usd_sensitivity:+.1f}% from baseline')
print(f'  KHR CRI changed by {khr_sensitivity:+.1f}% from baseline')
if abs(khr_sensitivity) > abs(usd_sensitivity):
    print(f'\n  → KHR segment is MORE fragile under stress.')
else:
    print(f'\n  → USD segment is MORE fragile under stress.')
print(f'═══════════════════════════════════════════════════════')

---
## Summary

Stress testing reveals how credit risk responds to hypothetical spread shocks. The OU parameters (particularly σ) scale with the shock, and the crisis probability shifts accordingly. This framework can help regulators assess vulnerability under adverse scenarios.